# Getting Started with ROS2 and Gazebo

Welcome to your ROS2 and Gazebo learning environment!

## Access Points
- **Gazebo GUI**: Access via noVNC at http://localhost:6080/vnc.html
- **This Notebook**: You're here! Use this for coding and experiments.

## Quick Start Guide

### 1. Test ROS2 Installation

In [15]:
import subprocess
import os

# Source ROS2 environment
os.environ['ROS_DISTRO'] = 'jazzy'
result = subprocess.run(['bash', '-c', 'source /opt/ros/jazzy/setup.bash && ros2 --version'], 
                       capture_output=True, text=True)
print("ROS2 Version:")
print(result.stdout)

ROS2 Version:



### 2. Launch Simple Gazebo World

Run this to start a basic simulation. Then check the GUI via noVNC!

In [ ]:
# Launch Gazebo with simple world (runs in background)
launch_cmd = '''cd /home/jovyan && 
source /opt/ros/jazzy/setup.bash && 
DISPLAY=:1 gz sim /home/jovyan/gazebo_worlds/simple_world.sdf &'''

subprocess.Popen(['bash', '-c', launch_cmd])
print("✓ Gazebo launched! Check the GUI at http://localhost:6080/vnc.html")
print("📍 You should see a red box in the simulation")

### 3. List ROS2 Topics

See what topics are available from the running simulation:

In [ ]:
result = subprocess.run(['bash', '-c', 'source /opt/ros/jazzy/setup.bash && ros2 topic list'], 
                       capture_output=True, text=True)
print("Available ROS2 Topics:")
print(result.stdout)

### 4. Next Steps

🎯 **What to do next:**
1. Open the GUI at http://localhost:6080/vnc.html
2. Play around with the Gazebo interface
3. Try creating your own robot models
4. Experiment with ROS2 topics and services

📚 **Learning Resources:**
- [ROS2 Tutorials](https://docs.ros.org/en/jazzy/Tutorials.html)
- [Gazebo Tutorials](https://gazebosim.org/docs)

Happy learning! 🚀

In [3]:
import subprocess

# Try to source ROS 2 Jazzy setup and print the version
cmd = "source /opt/ros/jazzy/setup.bash && ros2 --version"
result = subprocess.run(["bash", "-c", cmd], capture_output=True, text=True)

if result.returncode == 0:
    print("✅ ROS 2 Jazzy is installed!")
    print("ROS 2 version:", result.stdout.strip())
else:
    print("❌ ROS 2 Jazzy not found or environment not set.")
    print("Error:", result.stderr.strip())


❌ ROS 2 Jazzy not found or environment not set.
Error: usage: ros2 [-h] [--use-python-default-buffering]
            Call `ros2 <command> -h` for more detailed usage. ...
ros2: error: unrecognized arguments: --version


In [4]:
import subprocess

# Source ROS 2 Jazzy environment and get the version
cmd = "source /opt/ros/jazzy/setup.bash && ros2 --help"
result = subprocess.run(["bash", "-c", cmd], capture_output=True, text=True)

if result.returncode == 0:
    print("✅ ROS 2 Jazzy is installed!")
    # You can extract the version from the package info
    pkg_cmd = "source /opt/ros/jazzy/setup.bash && ros2 pkg xml rclpy | grep '<version>'"
    pkg_result = subprocess.run(["bash", "-c", pkg_cmd], capture_output=True, text=True)
    print("ROS 2 core library version:", pkg_result.stdout.strip())
else:
    print("❌ ROS 2 Jazzy not found or environment not set.")
    print("Error:", result.stderr.strip())


✅ ROS 2 Jazzy is installed!
ROS 2 core library version: <version>7.1.4</version>


In [5]:
import subprocess
import time

# Helper to run shell commands with ROS 2 environment loaded
def ros2_cmd(command):
    return subprocess.run(
        f"source /opt/ros/jazzy/setup.bash && {command}",
        shell=True, executable="/bin/bash",
        capture_output=True, text=True
    )

print("🔍 Testing ROS 2 Jazzy installation and features...\n")

# 1️⃣ Check ROS 2 core command
result = ros2_cmd("ros2 --help")
if result.returncode != 0:
    print("❌ ROS 2 environment not found.")
    print(result.stderr)
else:
    print("✅ ROS 2 environment detected.")

# 2️⃣ Launch a test talker node in the background
print("\n▶ Starting talker node...")
talker_proc = subprocess.Popen(
    "source /opt/ros/jazzy/setup.bash && ros2 run demo_nodes_cpp talker",
    shell=True, executable="/bin/bash",
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)
time.sleep(3)  # Give it time to start

# 3️⃣ List active topics
topics = ros2_cmd("ros2 topic list")
print("\n📡 Active topics:")
print(topics.stdout.strip())

# 4️⃣ Listen to chatter topic for a few messages
print("\n👂 Listening to /chatter topic for 3 seconds...")
listener = ros2_cmd("timeout 3 ros2 topic echo /chatter")
print(listener.stdout.strip())

# 5️⃣ Clean up the talker node
talker_proc.terminate()
print("\n🛑 Talker node terminated.")

print("\n✅ Advanced ROS 2 check complete.")


🔍 Testing ROS 2 Jazzy installation and features...

✅ ROS 2 environment detected.

▶ Starting talker node...

📡 Active topics:
/parameter_events
/rosout

👂 Listening to /chatter topic for 3 seconds...

🛑 Talker node terminated.

✅ Advanced ROS 2 check complete.


In [6]:
ros2_cmd("ros2 pkg list | grep demo_nodes_cpp")


CompletedProcess(args='source /opt/ros/jazzy/setup.bash && ros2 pkg list | grep demo_nodes_cpp', returncode=1, stdout='', stderr='')

In [7]:
#!/usr/bin/env python3
"""
ROS 2 Jazzy Installation Checker for JupyterHub
Tests basic and advanced ROS 2 features to verify installation
"""

import subprocess
import sys
import os
import time
import threading
from typing import Dict, List, Tuple, Optional
import tempfile
import json

class ROS2Checker:
    def __init__(self):
        self.results = {}
        self.errors = []
        
    def run_command(self, cmd: str, timeout: int = 10, capture_output: bool = True) -> Tuple[bool, str, str]:
        """Execute a shell command with timeout"""
        try:
            if capture_output:
                result = subprocess.run(
                    cmd, shell=True, capture_output=True, text=True, timeout=timeout
                )
                return result.returncode == 0, result.stdout, result.stderr
            else:
                result = subprocess.run(cmd, shell=True, timeout=timeout)
                return result.returncode == 0, "", ""
        except subprocess.TimeoutExpired:
            return False, "", f"Command timed out after {timeout} seconds"
        except Exception as e:
            return False, "", str(e)

    def check_basic_installation(self) -> Dict[str, bool]:
        """Test basic ROS 2 installation"""
        print("🔍 Testing ROS 2 Jazzy installation and features...")
        print("=" * 60)
        
        tests = {
            "ROS 2 Environment": self._check_ros2_env,
            "ROS 2 Jazzy Detection": self._check_jazzy_version,
            "Core ROS 2 Commands": self._check_core_commands,
            "Python rclpy": self._check_rclpy,
            "C++ rclcpp": self._check_rclcpp,
            "ROS 2 Topics": self._check_topics,
            "ROS 2 Services": self._check_services,
            "ROS 2 Parameters": self._check_parameters,
            "ROS 2 Actions": self._check_actions
        }
        
        results = {}
        for test_name, test_func in tests.items():
            print(f"\n📋 Testing {test_name}...")
            try:
                success = test_func()
                results[test_name] = success
                status = "✅ PASS" if success else "❌ FAIL"
                print(f"   {status}")
            except Exception as e:
                results[test_name] = False
                print(f"   ❌ FAIL - Exception: {e}")
                self.errors.append(f"{test_name}: {e}")
        
        return results

    def check_advanced_features(self) -> Dict[str, bool]:
        """Test advanced ROS 2 features"""
        print("\n" + "=" * 60)
        print("🚀 Testing Advanced ROS 2 Features...")
        print("=" * 60)
        
        advanced_tests = {
            "Lifecycle Nodes": self._check_lifecycle_nodes,
            "Component Loading": self._check_components,
            "Quality of Service (QoS)": self._check_qos,
            "ROS 2 Launch System": self._check_launch_system,
            "ROS 2 Bag Recording": self._check_ros2_bag,
            "ROS 2 Security": self._check_security_features,
            "Multi-Threaded Executors": self._check_executors,
            "ROS 2 Introspection": self._check_introspection,
            "Cross-Language Communication": self._check_cross_language,
            "ROS 2 Distributed Communication": self._check_distributed_comm
        }
        
        results = {}
        for test_name, test_func in advanced_tests.items():
            print(f"\n📋 Testing {test_name}...")
            try:
                success = test_func()
                results[test_name] = success
                status = "✅ PASS" if success else "❌ FAIL"
                print(f"   {status}")
            except Exception as e:
                results[test_name] = False
                print(f"   ❌ FAIL - Exception: {e}")
                self.errors.append(f"{test_name}: {e}")
        
        return results

    def _check_ros2_env(self) -> bool:
        """Check ROS 2 environment setup"""
        required_vars = ['ROS_VERSION', 'ROS_DISTRO', 'AMENT_PREFIX_PATH']
        missing_vars = []
        
        for var in required_vars:
            if var not in os.environ:
                missing_vars.append(var)
        
        if missing_vars:
            print(f"   Missing environment variables: {missing_vars}")
            return False
        
        if os.environ.get('ROS_VERSION') != '2':
            print(f"   Wrong ROS version: {os.environ.get('ROS_VERSION')}")
            return False
            
        if os.environ.get('ROS_DISTRO') != 'jazzy':
            print(f"   Wrong ROS distro: {os.environ.get('ROS_DISTRO')}")
            return False
            
        return True

    def _check_jazzy_version(self) -> bool:
        """Verify ROS 2 Jazzy specific version"""
        success, stdout, stderr = self.run_command("ros2 --version")
        if not success:
            print(f"   Command failed: {stderr}")
            return False
        
        if "jazzy" not in stdout.lower():
            print(f"   Jazzy not detected in version output: {stdout}")
            return False
            
        return True

    def _check_core_commands(self) -> bool:
        """Test core ROS 2 commands"""
        commands = [
            "ros2 --help",
            "ros2 pkg list",
            "ros2 node list",
            "ros2 topic list",
            "ros2 service list"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed command: {cmd} - {stderr}")
                return False
        
        return True

    def _check_rclpy(self) -> bool:
        """Test Python ROS 2 client library"""
        test_code = """
import rclpy
from rclpy.node import Node
from std_msgs.msg import String

try:
    rclpy.init()
    node = Node('test_node')
    pub = node.create_publisher(String, 'test_topic', 10)
    sub = node.create_subscription(String, 'test_topic', lambda msg: None, 10)
    node.destroy_node()
    rclpy.shutdown()
    print("SUCCESS")
except Exception as e:
    print(f"ERROR: {e}")
"""
        
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
            f.write(test_code)
            temp_file = f.name
        
        try:
            success, stdout, stderr = self.run_command(f"python3 {temp_file}")
            os.unlink(temp_file)
            
            if not success or "SUCCESS" not in stdout:
                print(f"   rclpy test failed: {stderr}")
                return False
            return True
        except Exception as e:
            os.unlink(temp_file)
            print(f"   Exception testing rclpy: {e}")
            return False

    def _check_rclcpp(self) -> bool:
        """Check if rclcpp is available (C++ client library)"""
        success, stdout, stderr = self.run_command("ros2 pkg list | grep rclcpp")
        return success and "rclcpp" in stdout

    def _check_topics(self) -> bool:
        """Test ROS 2 topics functionality"""
        # Test topic commands
        commands = [
            "ros2 topic list",
            "ros2 topic info /rosout",
            "ros2 topic type /rosout"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed: {cmd} - {stderr}")
                return False
        
        return True

    def _check_services(self) -> bool:
        """Test ROS 2 services functionality"""
        commands = [
            "ros2 service list",
            "ros2 service type /describe_parameters"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed: {cmd} - {stderr}")
                return False
        
        return True

    def _check_parameters(self) -> bool:
        """Test ROS 2 parameters functionality"""
        # Start a simple node in background and test parameters
        success, _, stderr = self.run_command("timeout 2 ros2 run demo_nodes_cpp talker &", timeout=3)
        time.sleep(1)
        
        param_success, _, param_stderr = self.run_command("ros2 param list", timeout=5)
        
        # Clean up any background processes
        self.run_command("pkill -f demo_nodes_cpp", timeout=2)
        
        return param_success

    def _check_actions(self) -> bool:
        """Test ROS 2 actions functionality"""
        commands = [
            "ros2 action list",
            "ros2 interface list | grep action"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed: {cmd} - {stderr}")
                return False
        
        return True

    def _check_lifecycle_nodes(self) -> bool:
        """Test lifecycle node functionality"""
        success, stdout, stderr = self.run_command("ros2 pkg list | grep lifecycle")
        return success and "lifecycle" in stdout

    def _check_components(self) -> bool:
        """Test component loading capability"""
        success, stdout, stderr = self.run_command("ros2 component types")
        return success

    def _check_qos(self) -> bool:
        """Test Quality of Service features"""
        qos_test = """
import rclpy
from rclpy.node import Node
from rclpy.qos import QoSProfile, ReliabilityPolicy, HistoryPolicy
from std_msgs.msg import String

try:
    rclpy.init()
    node = Node('qos_test_node')
    
    qos_profile = QoSProfile(
        reliability=ReliabilityPolicy.RELIABLE,
        history=HistoryPolicy.KEEP_LAST,
        depth=10
    )
    
    pub = node.create_publisher(String, 'qos_test_topic', qos_profile)
    node.destroy_node()
    rclpy.shutdown()
    print("SUCCESS")
except Exception as e:
    print(f"ERROR: {e}")
"""
        
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
            f.write(qos_test)
            temp_file = f.name
        
        try:
            success, stdout, stderr = self.run_command(f"python3 {temp_file}")
            os.unlink(temp_file)
            return success and "SUCCESS" in stdout
        except Exception:
            os.unlink(temp_file)
            return False

    def _check_launch_system(self) -> bool:
        """Test ROS 2 launch system"""
        commands = [
            "ros2 launch --help",
            "ros2 pkg list | grep launch"
        ]
        
        for cmd in commands:
            success, stdout, stderr = self.run_command(cmd, timeout=5)
            if not success:
                return False
        
        return "launch" in stdout

    def _check_ros2_bag(self) -> bool:
        """Test ROS 2 bag functionality"""
        commands = [
            "ros2 bag --help",
            "ros2 bag info --help"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed: {cmd} - {stderr}")
                return False
        
        return True

    def _check_security_features(self) -> bool:
        """Check ROS 2 security features availability"""
        success, stdout, stderr = self.run_command("ros2 security --help")
        return success

    def _check_executors(self) -> bool:
        """Test multi-threaded executor functionality"""
        executor_test = """
import rclpy
from rclpy.node import Node
from rclpy.executors import MultiThreadedExecutor
from std_msgs.msg import String

try:
    rclpy.init()
    node = Node('executor_test_node')
    executor = MultiThreadedExecutor()
    executor.add_node(node)
    node.destroy_node()
    rclpy.shutdown()
    print("SUCCESS")
except Exception as e:
    print(f"ERROR: {e}")
"""
        
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
            f.write(executor_test)
            temp_file = f.name
        
        try:
            success, stdout, stderr = self.run_command(f"python3 {temp_file}")
            os.unlink(temp_file)
            return success and "SUCCESS" in stdout
        except Exception:
            os.unlink(temp_file)
            return False

    def _check_introspection(self) -> bool:
        """Test ROS 2 introspection capabilities"""
        commands = [
            "ros2 interface list",
            "ros2 interface show std_msgs/msg/String"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed: {cmd} - {stderr}")
                return False
        
        return True

    def _check_cross_language(self) -> bool:
        """Check cross-language communication support"""
        # Check if both Python and C++ demo nodes are available
        py_success, _, _ = self.run_command("ros2 pkg list | grep demo_nodes_py")
        cpp_success, _, _ = self.run_command("ros2 pkg list | grep demo_nodes_cpp")
        
        return py_success and cpp_success

    def _check_distributed_comm(self) -> bool:
        """Test distributed communication features"""
        # Check DDS-related environment and tools
        dds_commands = [
            "ros2 daemon status",
            "ros2 doctor --report"
        ]
        
        success_count = 0
        for cmd in dds_commands:
            success, _, _ = self.run_command(cmd, timeout=10)
            if success:
                success_count += 1
        
        return success_count >= 1  # At least one DDS-related command should work

    def generate_report(self, basic_results: Dict[str, bool], advanced_results: Dict[str, bool]) -> None:
        """Generate a comprehensive test report"""
        print("\n" + "=" * 60)
        print("📊 ROS 2 JAZZY INSTALLATION REPORT")
        print("=" * 60)
        
        total_tests = len(basic_results) + len(advanced_results)
        passed_tests = sum(basic_results.values()) + sum(advanced_results.values())
        
        print(f"\n📈 Overall Results: {passed_tests}/{total_tests} tests passed")
        print(f"📈 Success Rate: {(passed_tests/total_tests)*100:.1f}%")
        
        print("\n🔍 Basic Installation Tests:")
        for test, result in basic_results.items():
            status = "✅" if result else "❌"
            print(f"   {status} {test}")
        
        print("\n🚀 Advanced Features Tests:")
        for test, result in advanced_results.items():
            status = "✅" if result else "❌"
            print(f"   {status} {test}")
        
        if self.errors:
            print("\n❌ Errors Encountered:")
            for error in self.errors:
                print(f"   • {error}")
        
        # Recommendations
        print("\n💡 Recommendations:")
        if passed_tests == total_tests:
            print("   🎉 Excellent! Your ROS 2 Jazzy installation is fully functional!")
            print("   🔥 All basic and advanced features are working correctly.")
        elif passed_tests >= total_tests * 0.8:
            print("   👍 Good installation! Most features are working.")
            print("   🔧 Consider fixing the failed tests for complete functionality.")
        elif passed_tests >= total_tests * 0.6:
            print("   ⚠️  Partial installation. Core features work but some advanced features are missing.")
            print("   📝 Check the failed tests and consider reinstalling missing packages.")
        else:
            print("   🚨 Installation has significant issues.")
            print("   🔄 Consider reinstalling ROS 2 Jazzy or checking your environment setup.")
        
        print("\n📚 For troubleshooting, visit: https://docs.ros.org/en/jazzy/")


def main():
    """Main function to run all tests"""
    print("🤖 ROS 2 Jazzy Installation Checker")
    print("🔍 Comprehensive testing of basic and advanced features")
    print("⏰ This may take a few minutes to complete...")
    
    checker = ROS2Checker()
    
    # Run basic tests
    basic_results = checker.check_basic_installation()
    
    # Run advanced tests
    advanced_results = checker.check_advanced_features()
    
    # Generate comprehensive report
    checker.generate_report(basic_results, advanced_results)
    
    return basic_results, advanced_results

if __name__ == "__main__":
    basic, advanced = main()

🤖 ROS 2 Jazzy Installation Checker
🔍 Comprehensive testing of basic and advanced features
⏰ This may take a few minutes to complete...
🔍 Testing ROS 2 Jazzy installation and features...

📋 Testing ROS 2 Environment...
   Missing environment variables: ['ROS_VERSION', 'AMENT_PREFIX_PATH']
   ❌ FAIL

📋 Testing ROS 2 Jazzy Detection...
   Command failed: Failed to load entry point 'security': Environment variable 'AMENT_PREFIX_PATH' is not set or empty
usage: ros2 [-h] [--use-python-default-buffering]
            Call `ros2 <command> -h` for more detailed usage. ...
ros2: error: unrecognized arguments: --version

   ❌ FAIL

📋 Testing Core ROS 2 Commands...
   Failed command: ros2 pkg list - Traceback (most recent call last):
  File "/opt/ros/jazzy/bin/ros2", line 33, in <module>
    sys.exit(load_entry_point('ros2cli==0.32.4', 'console_scripts', 'ros2')())
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/ros/jazzy/lib/python3.12/site-packages/ros2

In [8]:
#!/usr/bin/env python3
"""
ROS 2 Jazzy Installation Checker for JupyterHub
Tests basic and advanced ROS 2 features to verify installation
"""

import subprocess
import sys
import os
import time
import threading
from typing import Dict, List, Tuple, Optional
import tempfile
import json

class ROS2Checker:
    def __init__(self):
        self.results = {}
        self.errors = []
        
    def run_command(self, cmd: str, timeout: int = 10, capture_output: bool = True) -> Tuple[bool, str, str]:
        """Execute a shell command with timeout"""
        try:
            if capture_output:
                result = subprocess.run(
                    cmd, shell=True, capture_output=True, text=True, timeout=timeout
                )
                return result.returncode == 0, result.stdout, result.stderr
            else:
                result = subprocess.run(cmd, shell=True, timeout=timeout)
                return result.returncode == 0, "", ""
        except subprocess.TimeoutExpired:
            return False, "", f"Command timed out after {timeout} seconds"
        except Exception as e:
            return False, "", str(e)

    def check_basic_installation(self) -> Dict[str, bool]:
        """Test basic ROS 2 installation for JupyterLab Python environment"""
        print("🔍 Testing ROS 2 Jazzy installation and features for JupyterLab...")
        print("=" * 60)
        
        tests = {
            "ROS 2 Environment": self._check_ros2_env,
            "ROS 2 Jazzy Detection": self._check_jazzy_version,
            "Core ROS 2 Commands": self._check_core_commands,
            "Python rclpy": self._check_rclpy,
            "ROS 2 Topics": self._check_topics,
            "ROS 2 Services": self._check_services,
            "ROS 2 Parameters": self._check_parameters,
            "ROS 2 Actions": self._check_actions,
            "Python Message Types": self._check_python_msg_types
        }
        
        results = {}
        for test_name, test_func in tests.items():
            print(f"\n📋 Testing {test_name}...")
            try:
                success = test_func()
                results[test_name] = success
                status = "✅ PASS" if success else "❌ FAIL"
                print(f"   {status}")
            except Exception as e:
                results[test_name] = False
                print(f"   ❌ FAIL - Exception: {e}")
                self.errors.append(f"{test_name}: {e}")
        
        return results

    def check_advanced_features(self) -> Dict[str, bool]:
        """Test advanced ROS 2 features for JupyterLab Python environment"""
        print("\n" + "=" * 60)
        print("🚀 Testing Advanced ROS 2 Features for JupyterLab...")
        print("=" * 60)
        
        advanced_tests = {
            "Lifecycle Nodes (Python)": self._check_lifecycle_nodes,
            "Quality of Service (QoS)": self._check_qos,
            "ROS 2 Launch System": self._check_launch_system,
            "ROS 2 Bag Recording": self._check_ros2_bag,
            "Multi-Threaded Executors": self._check_executors,
            "ROS 2 Introspection": self._check_introspection,
            "Python Demo Nodes": self._check_python_demo_nodes,
            "ROS 2 Distributed Communication": self._check_distributed_comm,
            "Custom Message Creation": self._check_custom_messages,
            "ROS 2 Time and Clock": self._check_time_features
        }
        
        results = {}
        for test_name, test_func in advanced_tests.items():
            print(f"\n📋 Testing {test_name}...")
            try:
                success = test_func()
                results[test_name] = success
                status = "✅ PASS" if success else "❌ FAIL"
                print(f"   {status}")
            except Exception as e:
                results[test_name] = False
                print(f"   ❌ FAIL - Exception: {e}")
                self.errors.append(f"{test_name}: {e}")
        
        return results

    def _check_ros2_env(self) -> bool:
        """Check ROS 2 environment setup"""
        required_vars = ['ROS_VERSION', 'ROS_DISTRO', 'AMENT_PREFIX_PATH']
        missing_vars = []
        
        for var in required_vars:
            if var not in os.environ:
                missing_vars.append(var)
        
        if missing_vars:
            print(f"   Missing environment variables: {missing_vars}")
            return False
        
        if os.environ.get('ROS_VERSION') != '2':
            print(f"   Wrong ROS version: {os.environ.get('ROS_VERSION')}")
            return False
            
        if os.environ.get('ROS_DISTRO') != 'jazzy':
            print(f"   Wrong ROS distro: {os.environ.get('ROS_DISTRO')}")
            return False
            
        return True

    def _check_jazzy_version(self) -> bool:
        """Verify ROS 2 Jazzy specific version"""
        success, stdout, stderr = self.run_command("ros2 --version")
        if not success:
            print(f"   Command failed: {stderr}")
            return False
        
        if "jazzy" not in stdout.lower():
            print(f"   Jazzy not detected in version output: {stdout}")
            return False
            
        return True

    def _check_core_commands(self) -> bool:
        """Test core ROS 2 commands"""
        commands = [
            "ros2 --help",
            "ros2 pkg list",
            "ros2 node list",
            "ros2 topic list",
            "ros2 service list"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed command: {cmd} - {stderr}")
                return False
        
        return True

    def _check_rclpy(self) -> bool:
        """Test Python ROS 2 client library"""
        test_code = """
import rclpy
from rclpy.node import Node
from std_msgs.msg import String

try:
    rclpy.init()
    node = Node('test_node')
    pub = node.create_publisher(String, 'test_topic', 10)
    sub = node.create_subscription(String, 'test_topic', lambda msg: None, 10)
    node.destroy_node()
    rclpy.shutdown()
    print("SUCCESS")
except Exception as e:
    print(f"ERROR: {e}")
"""
        
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
            f.write(test_code)
            temp_file = f.name
        
        try:
            success, stdout, stderr = self.run_command(f"python3 {temp_file}")
            os.unlink(temp_file)
            
            if not success or "SUCCESS" not in stdout:
                print(f"   rclpy test failed: {stderr}")
                return False
            return True
        except Exception as e:
            os.unlink(temp_file)
            print(f"   Exception testing rclpy: {e}")
            return False

    def _check_rclcpp(self) -> bool:
        """Check if rclcpp is available (C++ client library)"""
        success, stdout, stderr = self.run_command("ros2 pkg list | grep rclcpp")
        return success and "rclcpp" in stdout

    def _check_topics(self) -> bool:
        """Test ROS 2 topics functionality"""
        # Test topic commands
        commands = [
            "ros2 topic list",
            "ros2 topic info /rosout",
            "ros2 topic type /rosout"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed: {cmd} - {stderr}")
                return False
        
        return True

    def _check_services(self) -> bool:
        """Test ROS 2 services functionality"""
        commands = [
            "ros2 service list",
            "ros2 service type /describe_parameters"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed: {cmd} - {stderr}")
                return False
        
        return True

    def _check_parameters(self) -> bool:
        """Test ROS 2 parameters functionality"""
        # Start a simple node in background and test parameters
        success, _, stderr = self.run_command("timeout 2 ros2 run demo_nodes_cpp talker &", timeout=3)
        time.sleep(1)
        
        param_success, _, param_stderr = self.run_command("ros2 param list", timeout=5)
        
        # Clean up any background processes
        self.run_command("pkill -f demo_nodes_cpp", timeout=2)
        
        return param_success

    def _check_actions(self) -> bool:
        """Test ROS 2 actions functionality"""
        commands = [
            "ros2 action list",
            "ros2 interface list | grep action"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed: {cmd} - {stderr}")
                return False
        
        return True

    def _check_lifecycle_nodes(self) -> bool:
        """Test lifecycle node functionality"""
        success, stdout, stderr = self.run_command("ros2 pkg list | grep lifecycle")
        return success and "lifecycle" in stdout

    def _check_components(self) -> bool:
        """Test component loading capability"""
        success, stdout, stderr = self.run_command("ros2 component types")
        return success

    def _check_qos(self) -> bool:
        """Test Quality of Service features"""
        qos_test = """
import rclpy
from rclpy.node import Node
from rclpy.qos import QoSProfile, ReliabilityPolicy, HistoryPolicy
from std_msgs.msg import String

try:
    rclpy.init()
    node = Node('qos_test_node')
    
    qos_profile = QoSProfile(
        reliability=ReliabilityPolicy.RELIABLE,
        history=HistoryPolicy.KEEP_LAST,
        depth=10
    )
    
    pub = node.create_publisher(String, 'qos_test_topic', qos_profile)
    node.destroy_node()
    rclpy.shutdown()
    print("SUCCESS")
except Exception as e:
    print(f"ERROR: {e}")
"""
        
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
            f.write(qos_test)
            temp_file = f.name
        
        try:
            success, stdout, stderr = self.run_command(f"python3 {temp_file}")
            os.unlink(temp_file)
            return success and "SUCCESS" in stdout
        except Exception:
            os.unlink(temp_file)
            return False

    def _check_launch_system(self) -> bool:
        """Test ROS 2 launch system"""
        commands = [
            "ros2 launch --help",
            "ros2 pkg list | grep launch"
        ]
        
        for cmd in commands:
            success, stdout, stderr = self.run_command(cmd, timeout=5)
            if not success:
                return False
        
        return "launch" in stdout

    def _check_ros2_bag(self) -> bool:
        """Test ROS 2 bag functionality"""
        commands = [
            "ros2 bag --help",
            "ros2 bag info --help"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed: {cmd} - {stderr}")
                return False
        
        return True

    def _check_security_features(self) -> bool:
        """Check ROS 2 security features availability"""
        success, stdout, stderr = self.run_command("ros2 security --help")
        return success

    def _check_executors(self) -> bool:
        """Test multi-threaded executor functionality"""
        executor_test = """
import rclpy
from rclpy.node import Node
from rclpy.executors import MultiThreadedExecutor
from std_msgs.msg import String

try:
    rclpy.init()
    node = Node('executor_test_node')
    executor = MultiThreadedExecutor()
    executor.add_node(node)
    node.destroy_node()
    rclpy.shutdown()
    print("SUCCESS")
except Exception as e:
    print(f"ERROR: {e}")
"""
        
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
            f.write(executor_test)
            temp_file = f.name
        
        try:
            success, stdout, stderr = self.run_command(f"python3 {temp_file}")
            os.unlink(temp_file)
            return success and "SUCCESS" in stdout
        except Exception:
            os.unlink(temp_file)
            return False

    def _check_introspection(self) -> bool:
        """Test ROS 2 introspection capabilities"""
        commands = [
            "ros2 interface list",
            "ros2 interface show std_msgs/msg/String"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed: {cmd} - {stderr}")
                return False
        
        return True

    def _check_cross_language(self) -> bool:
        """Check cross-language communication support"""
        # Check if both Python and C++ demo nodes are available
        py_success, _, _ = self.run_command("ros2 pkg list | grep demo_nodes_py")
        cpp_success, _, _ = self.run_command("ros2 pkg list | grep demo_nodes_cpp")
        
        return py_success and cpp_success

    def _check_distributed_comm(self) -> bool:
        """Test distributed communication features"""
        # Check DDS-related environment and tools
        dds_commands = [
            "ros2 daemon status",
            "ros2 doctor --report"
        ]
        
        success_count = 0
        for cmd in dds_commands:
            success, _, _ = self.run_command(cmd, timeout=10)
            if success:
                success_count += 1
        
        return success_count >= 1  # At least one DDS-related command should work

    def generate_report(self, basic_results: Dict[str, bool], advanced_results: Dict[str, bool]) -> None:
        """Generate a comprehensive test report"""
        print("\n" + "=" * 60)
        print("📊 ROS 2 JAZZY INSTALLATION REPORT")
        print("=" * 60)
        
        total_tests = len(basic_results) + len(advanced_results)
        passed_tests = sum(basic_results.values()) + sum(advanced_results.values())
        
        print(f"\n📈 Overall Results: {passed_tests}/{total_tests} tests passed")
        print(f"📈 Success Rate: {(passed_tests/total_tests)*100:.1f}%")
        
        print("\n🔍 Basic Installation Tests:")
        for test, result in basic_results.items():
            status = "✅" if result else "❌"
            print(f"   {status} {test}")
        
        print("\n🚀 Advanced Features Tests:")
        for test, result in advanced_results.items():
            status = "✅" if result else "❌"
            print(f"   {status} {test}")
        
        if self.errors:
            print("\n❌ Errors Encountered:")
            for error in self.errors:
                print(f"   • {error}")
        
        # Recommendations
        print("\n💡 Recommendations:")
        if passed_tests == total_tests:
            print("   🎉 Excellent! Your ROS 2 Jazzy installation is fully functional in JupyterLab!")
            print("   🐍 All Python-based ROS 2 features are working correctly.")
        elif passed_tests >= total_tests * 0.8:
            print("   👍 Good installation! Most Python ROS 2 features are working.")
            print("   🔧 Consider fixing the failed tests for complete functionality.")
        elif passed_tests >= total_tests * 0.6:
            print("   ⚠️  Partial installation. Core Python features work but some advanced features are missing.")
            print("   📝 Check the failed tests and consider installing missing Python packages.")
        else:
            print("   🚨 Installation has significant issues.")
            print("   🔄 Consider reinstalling ROS 2 Jazzy Python packages or checking your environment setup.")
        
        print("\n📚 For troubleshooting, visit: https://docs.ros.org/en/jazzy/")


def main():
    """Main function to run all tests"""
    print("🤖 ROS 2 Jazzy Installation Checker for JupyterLab")
    print("🐍 Comprehensive testing of Python-based ROS 2 features")
    print("⏰ This may take a few minutes to complete...")
    
    checker = ROS2Checker()
    
    # Run basic tests
    basic_results = checker.check_basic_installation()
    
    # Run advanced tests
    advanced_results = checker.check_advanced_features()
    
    # Generate comprehensive report
    checker.generate_report(basic_results, advanced_results)
    
    return basic_results, advanced_results

if __name__ == "__main__":
    basic, advanced = main()

🤖 ROS 2 Jazzy Installation Checker for JupyterLab
🐍 Comprehensive testing of Python-based ROS 2 features
⏰ This may take a few minutes to complete...
🔍 Testing ROS 2 Jazzy installation and features for JupyterLab...


AttributeError: 'ROS2Checker' object has no attribute '_check_python_msg_types'

In [9]:
#!/usr/bin/env python3
"""
ROS 2 Jazzy Installation Checker for JupyterHub
Tests basic and advanced ROS 2 features to verify installation
"""

import subprocess
import sys
import os
import time
import threading
from typing import Dict, List, Tuple, Optional
import tempfile
import json

class ROS2Checker:
    def __init__(self):
        self.results = {}
        self.errors = []
        
    def run_command(self, cmd: str, timeout: int = 10, capture_output: bool = True) -> Tuple[bool, str, str]:
        """Execute a shell command with timeout"""
        try:
            if capture_output:
                result = subprocess.run(
                    cmd, shell=True, capture_output=True, text=True, timeout=timeout
                )
                return result.returncode == 0, result.stdout, result.stderr
            else:
                result = subprocess.run(cmd, shell=True, timeout=timeout)
                return result.returncode == 0, "", ""
        except subprocess.TimeoutExpired:
            return False, "", f"Command timed out after {timeout} seconds"
        except Exception as e:
            return False, "", str(e)

    def check_basic_installation(self) -> Dict[str, bool]:
        """Test basic ROS 2 installation for JupyterLab Python environment"""
        print("🔍 Testing ROS 2 Jazzy installation and features for JupyterLab...")
        print("=" * 60)
        
        tests = {
            "ROS 2 Environment": self._check_ros2_env,
            "ROS 2 Jazzy Detection": self._check_jazzy_version,
            "Core ROS 2 Commands": self._check_core_commands,
            "Python rclpy": self._check_rclpy,
            "ROS 2 Topics": self._check_topics,
            "ROS 2 Services": self._check_services,
            "ROS 2 Parameters": self._check_parameters,
            "ROS 2 Actions": self._check_actions,
            "Python Message Types": self._check_python_msg_types
        }
        
        results = {}
        for test_name, test_func in tests.items():
            print(f"\n📋 Testing {test_name}...")
            try:
                success = test_func()
                results[test_name] = success
                status = "✅ PASS" if success else "❌ FAIL"
                print(f"   {status}")
            except Exception as e:
                results[test_name] = False
                print(f"   ❌ FAIL - Exception: {e}")
                self.errors.append(f"{test_name}: {e}")
        
        return results

    def check_advanced_features(self) -> Dict[str, bool]:
        """Test advanced ROS 2 features for JupyterLab Python environment"""
        print("\n" + "=" * 60)
        print("🚀 Testing Advanced ROS 2 Features for JupyterLab...")
        print("=" * 60)
        
        advanced_tests = {
            "Lifecycle Nodes (Python)": self._check_lifecycle_nodes,
            "Quality of Service (QoS)": self._check_qos,
            "ROS 2 Launch System": self._check_launch_system,
            "ROS 2 Bag Recording": self._check_ros2_bag,
            "Multi-Threaded Executors": self._check_executors,
            "ROS 2 Introspection": self._check_introspection,
            "Python Demo Nodes": self._check_python_demo_nodes,
            "ROS 2 Distributed Communication": self._check_distributed_comm,
            "Custom Message Creation": self._check_custom_messages,
            "ROS 2 Time and Clock": self._check_time_features
        }
        
        results = {}
        for test_name, test_func in advanced_tests.items():
            print(f"\n📋 Testing {test_name}...")
            try:
                success = test_func()
                results[test_name] = success
                status = "✅ PASS" if success else "❌ FAIL"
                print(f"   {status}")
            except Exception as e:
                results[test_name] = False
                print(f"   ❌ FAIL - Exception: {e}")
                self.errors.append(f"{test_name}: {e}")
        
        return results

    def _check_ros2_env(self) -> bool:
        """Check ROS 2 environment setup"""
        required_vars = ['ROS_VERSION', 'ROS_DISTRO', 'AMENT_PREFIX_PATH']
        missing_vars = []
        
        for var in required_vars:
            if var not in os.environ:
                missing_vars.append(var)
        
        if missing_vars:
            print(f"   Missing environment variables: {missing_vars}")
            return False
        
        if os.environ.get('ROS_VERSION') != '2':
            print(f"   Wrong ROS version: {os.environ.get('ROS_VERSION')}")
            return False
            
        if os.environ.get('ROS_DISTRO') != 'jazzy':
            print(f"   Wrong ROS distro: {os.environ.get('ROS_DISTRO')}")
            return False
            
        return True

    def _check_jazzy_version(self) -> bool:
        """Verify ROS 2 Jazzy specific version"""
        success, stdout, stderr = self.run_command("ros2 --version")
        if not success:
            print(f"   Command failed: {stderr}")
            return False
        
        if "jazzy" not in stdout.lower():
            print(f"   Jazzy not detected in version output: {stdout}")
            return False
            
        return True

    def _check_core_commands(self) -> bool:
        """Test core ROS 2 commands"""
        commands = [
            "ros2 --help",
            "ros2 pkg list",
            "ros2 node list",
            "ros2 topic list",
            "ros2 service list"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed command: {cmd} - {stderr}")
                return False
        
        return True

    def _check_rclpy(self) -> bool:
        """Test Python ROS 2 client library"""
        test_code = """
import rclpy
from rclpy.node import Node
from std_msgs.msg import String

try:
    rclpy.init()
    node = Node('test_node')
    pub = node.create_publisher(String, 'test_topic', 10)
    sub = node.create_subscription(String, 'test_topic', lambda msg: None, 10)
    node.destroy_node()
    rclpy.shutdown()
    print("SUCCESS")
except Exception as e:
    print(f"ERROR: {e}")
"""
        
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
            f.write(test_code)
            temp_file = f.name
        
        try:
            success, stdout, stderr = self.run_command(f"python3 {temp_file}")
            os.unlink(temp_file)
            
            if not success or "SUCCESS" not in stdout:
                print(f"   rclpy test failed: {stderr}")
                return False
            return True
        except Exception as e:
            os.unlink(temp_file)
            print(f"   Exception testing rclpy: {e}")
            return False

    def _check_python_msg_types(self) -> bool:
        """Test Python message type imports"""
        test_code = """
try:
    from std_msgs.msg import String, Int32, Float64, Bool
    from geometry_msgs.msg import Twist, Point, Quaternion
    from sensor_msgs.msg import Image, LaserScan
    print("SUCCESS")
except ImportError as e:
    print(f"ERROR: {e}")
"""
        
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
            f.write(test_code)
            temp_file = f.name
        
        try:
            success, stdout, stderr = self.run_command(f"python3 {temp_file}")
            os.unlink(temp_file)
            return success and "SUCCESS" in stdout
        except Exception:
            os.unlink(temp_file)
            return False

    def _check_topics(self) -> bool:
        """Test ROS 2 topics functionality"""
        # Test topic commands
        commands = [
            "ros2 topic list",
            "ros2 topic info /rosout",
            "ros2 topic type /rosout"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed: {cmd} - {stderr}")
                return False
        
        return True

    def _check_services(self) -> bool:
        """Test ROS 2 services functionality"""
        commands = [
            "ros2 service list",
            "ros2 service type /describe_parameters"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed: {cmd} - {stderr}")
                return False
        
        return True

    def _check_parameters(self) -> bool:
        """Test ROS 2 parameters functionality"""
        # Test parameter commands without relying on C++ demo nodes
        param_success, _, param_stderr = self.run_command("ros2 param list", timeout=5)
        
        if not param_success:
            print(f"   ros2 param list failed: {param_stderr}")
        
        return param_success

    def _check_actions(self) -> bool:
        """Test ROS 2 actions functionality"""
        commands = [
            "ros2 action list",
            "ros2 interface list | grep action"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed: {cmd} - {stderr}")
                return False
        
        return True

    def _check_lifecycle_nodes(self) -> bool:
        """Test lifecycle node functionality"""
        success, stdout, stderr = self.run_command("ros2 pkg list | grep lifecycle")
        return success and "lifecycle" in stdout

    def _check_python_demo_nodes(self) -> bool:
        """Check if Python demo nodes are available"""
        success, stdout, stderr = self.run_command("ros2 pkg list | grep demo_nodes_py")
        return success and "demo_nodes_py" in stdout

    def _check_qos(self) -> bool:
        """Test Quality of Service features"""
        qos_test = """
import rclpy
from rclpy.node import Node
from rclpy.qos import QoSProfile, ReliabilityPolicy, HistoryPolicy
from std_msgs.msg import String

try:
    rclpy.init()
    node = Node('qos_test_node')
    
    qos_profile = QoSProfile(
        reliability=ReliabilityPolicy.RELIABLE,
        history=HistoryPolicy.KEEP_LAST,
        depth=10
    )
    
    pub = node.create_publisher(String, 'qos_test_topic', qos_profile)
    node.destroy_node()
    rclpy.shutdown()
    print("SUCCESS")
except Exception as e:
    print(f"ERROR: {e}")
"""
        
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
            f.write(qos_test)
            temp_file = f.name
        
        try:
            success, stdout, stderr = self.run_command(f"python3 {temp_file}")
            os.unlink(temp_file)
            return success and "SUCCESS" in stdout
        except Exception:
            os.unlink(temp_file)
            return False

    def _check_launch_system(self) -> bool:
        """Test ROS 2 launch system"""
        commands = [
            "ros2 launch --help",
            "ros2 pkg list | grep launch"
        ]
        
        for cmd in commands:
            success, stdout, stderr = self.run_command(cmd, timeout=5)
            if not success:
                return False
        
        return "launch" in stdout

    def _check_ros2_bag(self) -> bool:
        """Test ROS 2 bag functionality"""
        commands = [
            "ros2 bag --help",
            "ros2 bag info --help"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed: {cmd} - {stderr}")
                return False
        
        return True

    def _check_custom_messages(self) -> bool:
        """Test ability to work with custom message types"""
        # Test if we can import and inspect message structures
        test_code = """
try:
    import rclpy
    from std_msgs.msg import String
    from geometry_msgs.msg import Twist
    
    # Test message creation and field access
    msg = String()
    msg.data = "test"
    
    twist = Twist()
    twist.linear.x = 1.0
    twist.angular.z = 0.5
    
    print("SUCCESS")
except Exception as e:
    print(f"ERROR: {e}")
"""
        
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
            f.write(test_code)
            temp_file = f.name
        
        try:
            success, stdout, stderr = self.run_command(f"python3 {temp_file}")
            os.unlink(temp_file)
            return success and "SUCCESS" in stdout
        except Exception:
            os.unlink(temp_file)
            return False

    def _check_executors(self) -> bool:
        """Test multi-threaded executor functionality"""
        executor_test = """
import rclpy
from rclpy.node import Node
from rclpy.executors import MultiThreadedExecutor
from std_msgs.msg import String

try:
    rclpy.init()
    node = Node('executor_test_node')
    executor = MultiThreadedExecutor()
    executor.add_node(node)
    node.destroy_node()
    rclpy.shutdown()
    print("SUCCESS")
except Exception as e:
    print(f"ERROR: {e}")
"""
        
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
            f.write(executor_test)
            temp_file = f.name
        
        try:
            success, stdout, stderr = self.run_command(f"python3 {temp_file}")
            os.unlink(temp_file)
            return success and "SUCCESS" in stdout
        except Exception:
            os.unlink(temp_file)
            return False

    def _check_introspection(self) -> bool:
        """Test ROS 2 introspection capabilities"""
        commands = [
            "ros2 interface list",
            "ros2 interface show std_msgs/msg/String"
        ]
        
        for cmd in commands:
            success, _, stderr = self.run_command(cmd, timeout=5)
            if not success:
                print(f"   Failed: {cmd} - {stderr}")
                return False
        
        return True

    def _check_time_features(self) -> bool:
        """Test ROS 2 time and clock functionality"""
        test_code = """
try:
    import rclpy
    from rclpy.node import Node
    from rclpy.time import Time
    from rclpy.duration import Duration
    from rclpy.clock import Clock, ClockType
    
    rclpy.init()
    node = Node('time_test_node')
    
    # Test time operations
    now = node.get_clock().now()
    duration = Duration(seconds=1.0)
    future_time = now + duration
    
    # Test different clock types
    steady_clock = Clock(clock_type=ClockType.STEADY_TIME)
    
    node.destroy_node()
    rclpy.shutdown()
    print("SUCCESS")
except Exception as e:
    print(f"ERROR: {e}")
"""
        
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
            f.write(test_code)
            temp_file = f.name
        
        try:
            success, stdout, stderr = self.run_command(f"python3 {temp_file}")
            os.unlink(temp_file)
            return success and "SUCCESS" in stdout
        except Exception:
            os.unlink(temp_file)
            return False

    def _check_distributed_comm(self) -> bool:
        """Test distributed communication features"""
        # Check DDS-related environment and tools
        dds_commands = [
            "ros2 daemon status",
            "ros2 doctor --report"
        ]
        
        success_count = 0
        for cmd in dds_commands:
            success, _, _ = self.run_command(cmd, timeout=10)
            if success:
                success_count += 1
        
        return success_count >= 1  # At least one DDS-related command should work

    def generate_report(self, basic_results: Dict[str, bool], advanced_results: Dict[str, bool]) -> None:
        """Generate a comprehensive test report"""
        print("\n" + "=" * 60)
        print("📊 ROS 2 JAZZY INSTALLATION REPORT")
        print("=" * 60)
        
        total_tests = len(basic_results) + len(advanced_results)
        passed_tests = sum(basic_results.values()) + sum(advanced_results.values())
        
        print(f"\n📈 Overall Results: {passed_tests}/{total_tests} tests passed")
        print(f"📈 Success Rate: {(passed_tests/total_tests)*100:.1f}%")
        
        print("\n🔍 Basic Installation Tests:")
        for test, result in basic_results.items():
            status = "✅" if result else "❌"
            print(f"   {status} {test}")
        
        print("\n🚀 Advanced Features Tests:")
        for test, result in advanced_results.items():
            status = "✅" if result else "❌"
            print(f"   {status} {test}")
        
        if self.errors:
            print("\n❌ Errors Encountered:")
            for error in self.errors:
                print(f"   • {error}")
        
        # Recommendations
        print("\n💡 Recommendations:")
        if passed_tests == total_tests:
            print("   🎉 Excellent! Your ROS 2 Jazzy installation is fully functional in JupyterLab!")
            print("   🐍 All Python-based ROS 2 features are working correctly.")
        elif passed_tests >= total_tests * 0.8:
            print("   👍 Good installation! Most Python ROS 2 features are working.")
            print("   🔧 Consider fixing the failed tests for complete functionality.")
        elif passed_tests >= total_tests * 0.6:
            print("   ⚠️  Partial installation. Core Python features work but some advanced features are missing.")
            print("   📝 Check the failed tests and consider installing missing Python packages.")
        else:
            print("   🚨 Installation has significant issues.")
            print("   🔄 Consider reinstalling ROS 2 Jazzy Python packages or checking your environment setup.")
        
        print("\n📚 For troubleshooting, visit: https://docs.ros.org/en/jazzy/")


def main():
    """Main function to run all tests"""
    print("🤖 ROS 2 Jazzy Installation Checker for JupyterLab")
    print("🐍 Comprehensive testing of Python-based ROS 2 features")
    print("⏰ This may take a few minutes to complete...")
    
    checker = ROS2Checker()
    
    # Run basic tests
    basic_results = checker.check_basic_installation()
    
    # Run advanced tests
    advanced_results = checker.check_advanced_features()
    
    # Generate comprehensive report
    checker.generate_report(basic_results, advanced_results)
    
    return basic_results, advanced_results

if __name__ == "__main__":
    basic, advanced = main()

🤖 ROS 2 Jazzy Installation Checker for JupyterLab
🐍 Comprehensive testing of Python-based ROS 2 features
⏰ This may take a few minutes to complete...
🔍 Testing ROS 2 Jazzy installation and features for JupyterLab...

📋 Testing ROS 2 Environment...
   Missing environment variables: ['ROS_VERSION', 'AMENT_PREFIX_PATH']
   ❌ FAIL

📋 Testing ROS 2 Jazzy Detection...
   Command failed: Failed to load entry point 'security': Environment variable 'AMENT_PREFIX_PATH' is not set or empty
usage: ros2 [-h] [--use-python-default-buffering]
            Call `ros2 <command> -h` for more detailed usage. ...
ros2: error: unrecognized arguments: --version

   ❌ FAIL

📋 Testing Core ROS 2 Commands...
   Failed command: ros2 pkg list - Traceback (most recent call last):
  File "/opt/ros/jazzy/bin/ros2", line 33, in <module>
    sys.exit(load_entry_point('ros2cli==0.32.4', 'console_scripts', 'ros2')())
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/ros/jazzy/lib

In [10]:
# Import ROS 2 Python client libraries
import rclpy
from rclpy.node import Node

# Check ROS 2 package versions
print(f"ROS 2 Version: {rclpy.__version__}")

# Initialize ROS 2 context
rclpy.init()

# Create a simple node to test communication with ROS graph
class TestNode(Node):
    def __init__(self):
        super().__init__('jazzy_test_node')
        self.get_logger().info('Test node initialized!')

# Instantiate and spin the node briefly
test_node = TestNode()
rclpy.spin_once(test_node)

# Clean up
test_node.destroy_node()
rclpy.shutdown()

# Verify environment configuration
import os
print("\nEnvironment Variables:")
print(f"ROS_DISTRO: {os.getenv('ROS_DISTRO', 'Not set')}")
print(f"PYTHONPATH: {os.getenv('PYTHONPATH', 'Not set')}")

AttributeError: module 'rclpy' has no attribute '__version__'

In [11]:
%%bash
# First verify ROS 2 Jazzy is properly installed (run in shell)
echo "=== ROS 2 Jazzy Verification ==="
ls /opt/ros/jazzy 2>/dev/null || echo "ERROR: ROS 2 Jazzy not installed!"
source /opt/ros/jazzy/setup.bash 2>/dev/null && echo "ROS 2 sourced successfully"

# Check Python compatibility (Jazzy requires Python 3.12+)
python3 -c "import sys; print(f'Python {sys.version_info.major}.{sys.version_info.minor} detected')"

# Install missing ROS 2 Python packages if needed
pip install -q rclpy rcl 2>/dev/null

# Verify ROS 2 Python bindings
python3 -c "import rclpy; print(f'rclpy version: {rclpy.__version__}')"

=== ROS 2 Jazzy Verification ===
_local_setup_util.py
bin
include
lib
local_setup.bash
local_setup.sh
local_setup.zsh
opt
setup.bash
setup.sh
setup.zsh
share
src
tools
ROS 2 sourced successfully
Python 3.12 detected


Traceback (most recent call last):
  File "<string>", line 1, in <module>
AttributeError: module 'rclpy' has no attribute '__version__'


CalledProcessError: Command 'b'# First verify ROS 2 Jazzy is properly installed (run in shell)\necho "=== ROS 2 Jazzy Verification ==="\nls /opt/ros/jazzy 2>/dev/null || echo "ERROR: ROS 2 Jazzy not installed!"\nsource /opt/ros/jazzy/setup.bash 2>/dev/null && echo "ROS 2 sourced successfully"\n\n# Check Python compatibility (Jazzy requires Python 3.12+)\npython3 -c "import sys; print(f\'Python {sys.version_info.major}.{sys.version_info.minor} detected\')"\n\n# Install missing ROS 2 Python packages if needed\npip install -q rclpy rcl 2>/dev/null\n\n# Verify ROS 2 Python bindings\npython3 -c "import rclpy; print(f\'rclpy version: {rclpy.__version__}\')"\n'' returned non-zero exit status 1.

In [12]:
# Now run the Python test (paste this in a NEW cell after the bash cell)
import os
import sys
import rclpy
from rclpy.node import Node

# Critical: Source ROS 2 environment programmatically
os.environ["ROS_DISTRO"] = "jazzy"
os.environ["AMENT_PREFIX_PATH"] = "/opt/ros/jazzy"
os.environ["LD_LIBRARY_PATH"] = "/opt/ros/jazzy/lib"
os.environ["PYTHONPATH"] = "/opt/ros/jazzy/local/lib/python3.12/dist-packages:/opt/ros/jazzy/lib/python3.12/site-packages"

print(f"Python version: {sys.version}")
print(f"ROS 2 Jazzy environment variables set:\nROS_DISTRO={os.getenv('ROS_DISTRO')}\nPYTHONPATH={os.getenv('PYTHONPATH')}")

try:
    # Initialize ROS 2
    rclpy.init(args=None)
    
    # Create test node
    class JazzyTestNode(Node):
        def __init__(self):
            super().__init__('jupyter_test_node')
            self.get_logger().info('✅ ROS 2 Jazzy node initialized in JupyterLab!')
            self.get_logger().info(f'Using rclpy v{rclpy.__version__}')
    
    node = JazzyTestNode()
    
    # Spin once to verify communication
    rclpy.spin_once(node, timeout_sec=0.1)
    
    # Cleanup
    node.destroy_node()
    rclpy.shutdown()
    print("\n✅ ROS 2 Jazzy test completed successfully!")

except Exception as e:
    print(f"\n❌ TEST FAILED: {type(e).__name__}")
    print(f"Error details: {str(e)}")
    print("\nDebug suggestions:")
    print("1. Ensure ROS 2 Jazzy is installed: sudo apt install ros-jazzy-desktop")
    print("2. Verify Python 3.12: python3 --version")
    print("3. Check paths: ls /opt/ros/jazzy/lib/python3.12/site-packages | grep rclpy")

Python version: 3.12.3 (main, Jun 18 2025, 17:59:45) [GCC 13.3.0]
ROS 2 Jazzy environment variables set:
ROS_DISTRO=jazzy
PYTHONPATH=/opt/ros/jazzy/local/lib/python3.12/dist-packages:/opt/ros/jazzy/lib/python3.12/site-packages

❌ TEST FAILED: AttributeError
Error details: module 'rclpy' has no attribute '__version__'

Debug suggestions:
1. Ensure ROS 2 Jazzy is installed: sudo apt install ros-jazzy-desktop
2. Verify Python 3.12: python3 --version
3. Check paths: ls /opt/ros/jazzy/lib/python3.12/site-packages | grep rclpy


[INFO] [1754667516.081385171] [jupyter_test_node]: ✅ ROS 2 Jazzy node initialized in JupyterLab!


In [13]:
%%bash
# First: Verify ROS 2 installation without version assumptions
echo "ROS 2 Installation Check:"
ls /opt/ros || echo "ERROR: ROS 2 not installed in /opt/ros!"

# Find actual ROS 2 version (works for any distro)
ROS_DISTRO=$(grep -r "source /opt/ros/" /etc/profile.d/ 2>/dev/null | cut -d'/' -f4 | head -1)
if [ -z "$ROS_DISTRO" ]; then
    ROS_DISTRO=$(basename $(ls -d /opt/ros/* 2>/dev/null | head -1))
fi
echo "Detected ROS 2 distro: $ROS_DISTRO"

# Find correct Python version used by ROS
ROS_PYTHON_VERSION=$(ls /opt/ros/$ROS_DISTRO/lib | grep python | sed 's/python//')
if [ -z "$ROS_PYTHON_VERSION" ]; then
    ROS_PYTHON_VERSION="3.10"  # Fallback for older distros
fi
echo "ROS 2 Python version: $ROS_PYTHON_VERSION"

# Update PYTHONPATH dynamically
echo "export PYTHONPATH=/opt/ros/$ROS_DISTRO/lib/python$ROS_PYTHON_VERSION/site-packages:\$PYTHONPATH" > /tmp/ros_env.sh
source /tmp/ros_env.sh
rm /tmp/ros_env.sh

# Verify rclpy exists for this Python version
python3 -c "import rclpy; print(f'rclpy found for Python {\".\".join(map(str, __import__('sys').version_info[:2]))}')" 2>/dev/null || echo "ERROR: rclpy not installed for this Python version"

ROS 2 Installation Check:
jazzy
Detected ROS 2 distro: jazzy
ROS 2 Python version: 3.12
rclpy found for Python 3.12


In [ ]:
%%bash
# First verify environment and install any missing components
echo "🔍 Advanced ROS 2 Jazzy Feature Verification"
echo "------------------------------------------"

# Detect ROS 2 version and Python compatibility
ROS_DISTRO=$(grep -r "source /opt/ros/" /etc/profile.d/ 2>/dev/null | cut -d'/' -f4 | head -1)
if [ -z "$ROS_DISTRO" ]; then
    ROS_DISTRO=$(basename $(ls -d /opt/ros/* 2>/dev/null | head -1))
fi

echo "Detected ROS 2 distro: $ROS_DISTRO"
echo "Required for Jazzy: Python 3.12+"

# Check Python version compatibility
python3 -c "import sys; 
if sys.version_info < (3,12):
    print(f'⚠️ WARNING: Python {sys.version_info.major}.{sys.version_info.minor} detected - Jazzy requires 3.12+');
else:
    print('✅ Python 3.12+ detected - compatible with Jazzy')"

# Install Jazzy-specific packages if needed
if [ "$ROS_DISTRO" = "jazzy" ]; then
    echo -e "\n📦 Installing Jazzy advanced feature dependencies..."
    sudo apt-get update > /dev/null 2>&1
    sudo apt-get install -y ros-jazzy-ros2launch ros-jazzy-rqt-common-plugins > /dev/null 2>&1
    pip install -q rclpy rclpy-components rclpy-action rclpy-lifecycle
fi

echo -e "\n🔍 Verifying critical Jazzy components:"
ls /opt/ros/jazzy/share/rclpy 2>/dev/null && echo "✅ rclpy installed"
ls /opt/ros/jazzy/share/rclcpp_components 2>/dev/null && echo "✅ rclcpp_components installed"
ls /opt/ros/jazzy/share/action_tutorials 2>/dev/null && echo "✅ action_tutorials installed (for action testing)"

🔍 Advanced ROS 2 Jazzy Feature Verification
------------------------------------------
Detected ROS 2 distro: jazzy
Required for Jazzy: Python 3.12+
✅ Python 3.12+ detected - compatible with Jazzy

📦 Installing Jazzy advanced feature dependencies...
